In [1]:
import subprocess, sys, os
import cml.workers_v1 as workers

DASHBOARD_PORT = os.environ['CDSW_READONLY_PORT']
DASHBOARD_IP = os.environ['CDSW_IP_ADDRESS']

command = "ray start --head --block --include-dashboard=true --dashboard-port=$CDSW_READONLY_PORT --num-gpus=0 &" 

subprocess.run(command, shell = True, executable="/bin/bash")

with open("RAY_HEAD_IP", 'w') as output_file:
    output_file.write(DASHBOARD_IP)
            
ray_head_addr = DASHBOARD_IP + ':6379'
ray_url = f"ray://{DASHBOARD_IP}:10001" 
worker_start_cmd = f"!ray start --block --address={ray_head_addr}"

ray_workers = workers.launch_workers(
    n=5, 
    cpu=1, 
    memory=16,
    nvidia_gpu=0,
    code=worker_start_cmd,
)

Skipping addon with invalid or excluded ID: {'type': 'cmladdon', 'path': '/runtime-addons/cmladdon-2.0.49-b279', 'spec': '\nenv:\n  MLFLOW_TRACKING_URI: cml://localhost\n  MLFLOW_REGISTRY_URI: cml://localhost\n  PYTHONPATH: ${PYTHONPATH}:/opt/cmladdons/python/site-customize\n  R_LIBS_SITE: ${R_LIBS_SITE}:/opt/cmladdons/r/libs\npaths:\n  - /opt/cmladdons', 'version': '', 'id': -1}
2025-07-15 11:07:44,460	INFO usage_lib.py:467 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.
2025-07-15 11:07:44,460	INFO scripts.py:971 -- Local node IP: 10.42.1.93
2025-07-15 11:07:56,667	SUCC scripts.py:1007 -- --------------------
2025-07-15 11:07:56,668	SUCC scripts.py:1008 -- Ray

In [2]:
import ray
from ray.data import Dataset
from ray.train import ScalingConfig
from ray.train.xgboost import XGBoostTrainer
import xgboost as xgb
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import time
import os

from ray.data import Dataset

def calculate_all_features_for_group(group_df: pd.DataFrame) -> pd.DataFrame:
    """Calculates aggregated features for a single user (a group of records)."""
    group_df['is_fraud'] = group_df['is_fraud'].astype(bool)
    nocturnal_hours = (group_df['hour_of_day'] >= 22) | (group_df['hour_of_day'] <= 6)
    features = {
        'total_calls': len(group_df),
        'outgoing_call_ratio': (group_df['call_direction'] == 'outgoing').mean(),
        'avg_duration': group_df['duration'].mean(),
        'std_duration': group_df['duration'].std(),
        'nocturnal_call_ratio': nocturnal_hours.mean(),
        'mobility': group_df['cell_tower'].nunique(),
        'is_fraud': group_df['is_fraud'].iloc[0]
    }
    return pd.DataFrame([features], index=[group_df['msisdn'].iloc[0]])

def feature_engineering_ray(ds: ray.data.Dataset) -> ray.data.Dataset:
    """Performs feature engineering on the raw call data using Ray Data."""
    print("Performing feature engineering...")
    user_features_ds = ds.groupby('msisdn').map_groups(
        calculate_all_features_for_group
    )
    return user_features_ds

def prepare_data(dataset: Dataset) -> tuple[Dataset, Dataset, Dataset]:
    """Splits the dataset into train (70%), validation (15%), and test (15%) sets."""
    print("Splitting data into training, validation, and test sets...")
    seed = 42
    train_dataset, rest = dataset.train_test_split(test_size=0.3, shuffle=True, seed=seed)
    valid_dataset, test_dataset = rest.train_test_split(test_size=0.5, shuffle=True, seed=seed)
    
    print(f"Train set size: {train_dataset.count()}")
    print(f"Validation set size: {valid_dataset.count()}")
    print(f"Test set size: {test_dataset.count()}")
    
    return train_dataset, valid_dataset, test_dataset

def train_fraud_detection_model_xgb_ray(train_ds: ray.data.Dataset, valid_ds: ray.data.Dataset):
    """Trains a fraud detection model using XGBoost with Ray Train."""
    print("\nTraining the XGBoost model with Ray Train...")

    train_ds = train_ds.map_batches(lambda df: df.fillna(0), batch_format="pandas")
    valid_ds = valid_ds.map_batches(lambda df: df.fillna(0), batch_format="pandas")

    print("Calculating scale_pos_weight from training data for class imbalance...")
    num_fraud = train_ds.filter(lambda row: row["is_fraud"] == True).count()
    num_non_fraud = train_ds.filter(lambda row: row["is_fraud"] == False).count()

    if num_fraud > 0 and num_non_fraud > 0:
        scale_pos_weight = num_non_fraud / num_fraud
        print(f"scale_pos_weight determined to be: {scale_pos_weight:.2f}")
    else:
        scale_pos_weight = 1.0
        print(f"Warning: Insufficient classes to calculate scale_pos_weight. Defaulting to 1.0.")

    xgb_params = {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "tree_method": "hist",
        "scale_pos_weight": scale_pos_weight,
        "random_state": 42,
    }

    label_column = 'is_fraud'
    
    trainer = XGBoostTrainer(
        scaling_config=ScalingConfig(num_workers=5, use_gpu=False),
        label_column=label_column,
        params=xgb_params,
        datasets={"train": train_ds, "valid": valid_ds},
    )

    result = trainer.fit()
    print("\nModel Training Complete.")
    
    # Corrected: Use get_best_checkpoint method
    best_checkpoint = result.get_best_checkpoint(metric="valid-logloss", mode="min")
    
    if best_checkpoint:
        with best_checkpoint.as_directory() as checkpoint_dir:
            #model_path = os.path.join(checkpoint_dir, "model.xgb")
            model_path = os.path.join(checkpoint_dir, "model.ubj")
            if os.path.exists(model_path):
                booster = xgb.Booster()
                booster.load_model(model_path)
                return booster
    print("Could not load a model from checkpoint.")
    return None

def evaluate_model(booster: xgb.Booster, test_ds: ray.data.Dataset):
    """Evaluates the trained model on the unseen test dataset."""
    print("\n--- Model Evaluation on Unseen Test Data ---")

    test_ds = test_ds.map_batches(lambda df: df.fillna(0), batch_format="pandas")

    feature_columns = [col for col in test_ds.columns() if col != 'is_fraud']
    label_column = 'is_fraud'
    
    test_df = test_ds.to_pandas()
    
    if test_df.empty:
        print("Test dataset is empty. Cannot evaluate.")
        return

    X_test = test_df[feature_columns]
    y_test = test_df[label_column]
    dmatrix_test = xgb.DMatrix(X_test)
    
    y_pred_proba = booster.predict(dmatrix_test)
    y_pred = (y_pred_proba > 0.5).astype(int)

    print("\nConfusion Matrix (Test Data):")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report (Test Data):")
    print(classification_report(y_test, y_pred))

    feature_scores = booster.get_score(importance_type='weight')
    if feature_scores:
        feature_importances = pd.Series(feature_scores).sort_values(ascending=False)
        print("\nFeature Importances:")
        print(feature_importances)


if __name__ == '__main__':
    
    raw_data_filename = 'cdr_data.csv'
    model_output_filename = 'fraud_detection_model_xgb_ray.json'

    try:
        print(f"\nReading '{raw_data_filename}' with Ray Data...")
        raw_ds = ray.data.read_csv(
            raw_data_filename,
        )
    except Exception as e:
        print(f"Error reading raw data file: {e}")
        exit()

    start_time = time.time()

    # 1. Split data into three sets
    train_raw_ds, valid_raw_ds, test_raw_ds = prepare_data(raw_ds)

    # 2. Apply feature engineering to each split
    train_features_ds = feature_engineering_ray(train_raw_ds)
    valid_features_ds = feature_engineering_ray(valid_raw_ds)
    test_features_ds = feature_engineering_ray(test_raw_ds)

    # 3. Train the model
    fraud_model_booster = train_fraud_detection_model_xgb_ray(train_features_ds, valid_features_ds)

    # 4. Evaluate and save the final model
    if fraud_model_booster:
        evaluate_model(fraud_model_booster, test_features_ds)
        
        fraud_model_booster.save_model(model_output_filename)
        print(f"\nTrained XGBoost model saved to '{model_output_filename}'")
    else:
        print("Model training failed, so no evaluation or saving was performed.")

    print(f"\nProcess complete in {time.time() - start_time:.2f} seconds.")
    
    # ray.shutdown()

2025-07-15 11:09:40,990	INFO worker.py:1723 -- Connecting to existing Ray cluster at address: 10.42.1.93:6379...
2025-07-15 11:09:41,025	INFO worker.py:1908 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8100 



Reading 'cdr_data.csv' with Ray Data...


2025-07-15 11:09:51,089	INFO logging.py:295 -- Registered dataset logger for dataset dataset_2_0


Splitting data into training, validation, and test sets...


2025-07-15 11:09:51,367	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_2_0. Full logs are in /tmp/ray/session_2025-07-15_11-07-44_461720_157/logs/ray-data
2025-07-15 11:09:51,368	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_2_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AggregateNumRows[AggregateNumRows]


...

Shuffle Map 3:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Reduce 4:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

- MapBatches(calculate_all_features_for_group)->MapBatches(<lambda>) 5: 0.00 row [00:00, ? row/s]

2025-07-15 11:14:26,937	INFO streaming_executor.py:227 -- ✔️  Dataset dataset_32_0 execution finished in 4.46 seconds



Confusion Matrix (Test Data):
[[18981     0]
 [    0  1000]]

Classification Report (Test Data):
              precision    recall  f1-score   support

       False       1.00      1.00      1.00     18981
        True       1.00      1.00      1.00      1000

    accuracy                           1.00     19981
   macro avg       1.00      1.00      1.00     19981
weighted avg       1.00      1.00      1.00     19981


Feature Importances:
total_calls             9.0
outgoing_call_ratio     3.0
avg_duration            3.0
nocturnal_call_ratio    3.0
dtype: float64

Trained XGBoost model saved to 'fraud_detection_model_xgb_ray.json'

Process complete in 276.25 seconds.
